In [1]:
cd models/research/object_detection

/home/scar3crow/Dropbox/WorkStation-Subrata/python/models/research/object_detection


In [3]:
import numpy as np
import pandas as pd
import cv2
import os
import tqdm
from scipy.io import loadmat

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

from PIL import Image
import pytesseract

import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from keras import backend as K

from utils import *

from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.layers import *

from keras.applications import MobileNetV2
from keras.applications import InceptionResNetV2

from keras.models import Model
from keras.models import model_from_json


In [4]:
# Making a list of image path

inv_directory = '/home/scar3crow/Downloads/INVOICES/8-6-short'  ## 'invoices' is a zip file of jpg images in ...../Downloads 
                                                        
inv_image = ['/home/scar3crow/Downloads/INVOICES/8-6-short/{}'.format(i) for i in os.listdir(inv_directory)] # making the list
inv_image.sort() # Sorting the list

print('Number of images = ', len(inv_image))
inv_image[50]

Number of images =  62


'/home/scar3crow/Downloads/INVOICES/8-6-short/56a.jpg'

In [5]:
# Check sizes of exiting images :

# Create a Dataframe with image id and height(row) and width(column):

rows = []
columns = []
image_sl = []
df = pd.DataFrame()

for i in range(len(inv_image)):
    image = cv2.imread(inv_image[i]) ## Loading image
    height, width, _ = image.shape
    rows.append(height)
    columns.append(width)
    image_sl.append(inv_image[i])
    
row_values = pd.Series(rows)
col_values = pd.Series(columns)
image_num = pd.Series(image_sl)


df.insert(loc=0, column='image_serial', value=image_num)
df.insert(loc=1, column='rows', value=row_values)
df.insert(loc=2, column='columns', value=col_values)


print('max height = ', max(df['rows']))
print('max width = ', max(df['columns']))
df.tail(3)

max height =  326
max width =  554


,image_serial,rows,columns
59,/home/scar3crow/Downloads/INVOICES/8-6-short/7...,258,503
60,/home/scar3crow/Downloads/INVOICES/8-6-short/8...,169,510
61,/home/scar3crow/Downloads/INVOICES/8-6-short/9...,200,471


In [6]:
# Loading output of VGG Image Annotation tool and create a dataframe

r_1_data = pd.read_csv('/home/scar3crow/Downloads/invoice_data_1/short_via_data/1st_via_short.csv')
r_2_data = pd.read_csv('/home/scar3crow/Downloads/invoice_data_1/short_via_data/2nd_via_short.csv')
r_3_data = pd.read_csv('/home/scar3crow/Downloads/invoice_data_1/short_via_data/3rd_via_short.csv')

r_data = pd.concat([r_1_data, r_2_data, r_3_data], ignore_index=True, sort =False)

num_obj = r_data['region_count'][0] # number of objects in each photo
r_data.drop(r_data.columns[[1, 2, 3, 4]], axis=1, inplace=True) # reduce unnecessary columns
r_data.rename({'#filename': 'img_id'}, axis=1, inplace=True) # changing column name
num_images = r_data["img_id"].nunique() # Find out number of unique images

print('Number of classes = ', num_obj)
print('Number of unique images = ', num_images)
r_data.head()

## Remove all rows pertaining to 63a.jpg and 64.jpg since these images have been kept in test folder

id_63a = r_data.index[r_data['img_id'] == '63a.jpg'].tolist() # Finding the index
id_63a_64a = id_63a + r_data.index[r_data['img_id'] == '64a.jpg'].tolist() # Finding the index

r_data = r_data.drop(r_data.index[id_63a_64a]).reset_index(drop=True)

num_images = r_data["img_id"].nunique() # Find out number of unique images

print('Number of unique images = ', num_images)
r_data.tail(3)

r_data.tail(3)

Number of classes =  5
Number of unique images =  64
Number of unique images =  62


,img_id,region_shape_attributes,region_attributes
307,62a.jpg,"{""name"":""rect"",""x"":290,""y"":40,""width"":47,""heig...","{""text"":""invoice""}"
308,62a.jpg,"{""name"":""rect"",""x"":405,""y"":41,""width"":60,""heig...","{""text"":""inv_date""}"
309,62a.jpg,"{""name"":""rect"",""x"":290,""y"":103,""width"":71,""hei...","{""text"":""po""}"


In [7]:
# Making a dataframe for Image_id, x, y, width, height and class

x = []
y = []
width = []
height = []
obj_class = []
i_width = []
i_height = []


for i in range(len(r_data)):
    
    r_size = r_data.values[i, 1][1:(len(r_data.values[i, 1])-1)]
    r_size_par = r_size.split(",")
    
    x.append(int("".join(filter(str.isdigit, r_size_par[1]))))
    y.append(int("".join(filter(str.isdigit, r_size_par[2]))))
    width.append(int("".join(filter(str.isdigit, r_size_par[3]))))
    height.append(int("".join(filter(str.isdigit, r_size_par[4]))))
    
    r_attribs = r_data.values[i, 2][1:(len(r_data.values[i, 2])-1)]
    r_attribs_par = r_attribs.split(':')[1]
    obj_class.append(r_attribs_par[1:(len(r_attribs_par)-1)])
    
    foto_id = r_data['img_id'][i]
    img_path = '/home/scar3crow/Downloads/INVOICES/8-6-short/' + foto_id
    foto_index = df.index[df['image_serial'] == img_path]
    foto_width = df['columns'][foto_index].tolist()
    foto_height = df['rows'][foto_index].tolist()
    i_width.append(foto_width[0])
    i_height.append(foto_height[0])
    
x_values = pd.Series(x)
y_values = pd.Series(y)
width_values = pd.Series(width)
height_values = pd.Series(height)
class_values = pd.Series(obj_class)
i_width_values = pd.Series(i_width)
i_height_values = pd.Series(i_height)

r_data.insert(loc=1, column='x', value=x_values)
r_data.insert(loc=2, column='y', value=y_values)
r_data.insert(loc=3, column='box_width', value=width_values)
r_data.insert(loc=4, column='box_height', value=height_values)
r_data.insert(loc=5, column='obj_class', value=class_values)

r_data.insert(loc=8, column='image_width', value=i_width_values)
r_data.insert(loc=9, column='image_height', value=i_height_values)


r_data.drop(r_data.columns[[6, 7]], axis=1, inplace=True) # reduce unnecessary columns

r_data.tail(3)

,img_id,x,y,box_width,box_height,obj_class,image_width,image_height
307,62a.jpg,290,40,47,24,invoice,540,212
308,62a.jpg,405,41,60,24,inv_date,540,212
309,62a.jpg,290,103,71,28,po,540,212


In [8]:
print('Number of unique images = ', r_data['img_id'].nunique())  # print total no, of unique images

print('Number of classes in diff. categories = ', r_data['obj_class'].value_counts()) 

Number of unique images =  62
Number of classes in diff. categories =  po          62
inv_date    62
buyer       62
invoice     61
vendor      60
invoie       1
vndor        1
vendot       1
Name: obj_class, dtype: int64


In [9]:
# Correcting above wrong spelling of object classes and rechecking

id_1 = r_data.index[r_data['obj_class'] == 'vndor'] # Finding the index
id_2 = r_data.index[r_data['obj_class'] == 'invoie'] # Finding the index
id_3 = r_data.index[r_data['obj_class'] == 'vendot'] # Finding the index

r_data.at[id_1, 'obj_class'] = 'vendor' # writing the correct spelling 
r_data.at[id_2, 'obj_class'] = 'invoice' # # writing the correct spelling
r_data.at[id_3, 'obj_class'] = 'vendor' # # writing the correct spelling

print('Number of unique images = ', r_data['img_id'].nunique())  # print total no, of unique images
print('Number of unique classes = ', r_data['obj_class'].nunique())
print('Number of classes in diff. categories = ', r_data['obj_class'].value_counts()) 


Number of unique images =  62
Number of unique classes =  5
Number of classes in diff. categories =  invoice     62
po          62
inv_date    62
vendor      62
buyer       62
Name: obj_class, dtype: int64


In [10]:
categories = r_data['obj_class'].unique() # make a list of unique images

values = np.array(categories)
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)

onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
y_encoded = onehot_encoder.fit_transform(integer_encoded)

print(categories)                               
print(y_encoded)

['vendor' 'buyer' 'invoice' 'inv_date' 'po']
[[0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0.]]


In [32]:
o_class = categories[1]
o_class

'buyer'

In [34]:
id = categories.tolist().index(o_class)
type(id)

int

In [11]:
# Preparing Input(X) and Target(Y) file for training :

# Variable Definition

channels = 3

classes = 5 # vendor, buyer, invoice, inv_date, po
info = 5 + classes    # pc, x, y, h, w, and class probabilities

grid_y_axis = 14  # each image is to be segmented to 16 x 16 grid
grid_x_axis = 14  # # each image is to be segmented to 16 x 16 grid

img_w = 224 # target sizes as per MobileNetV2
img_h = 224 #target sizes as per MobileNetV2

# Start :

X_final = [] # X_final list to convert to np array later
Y_final = [] # Y_final list to convert to np array later

image_list = r_data['img_id'].unique() # make a list of unique images

for i in image_list:
    
    
    image_path = '/home/scar3crow/Downloads/INVOICES/8-6-short/' + i
    
    x = cv2.imread(image_path)
    x_ratio = img_w / x.shape[1]
    y_ratio = img_h / x.shape[0]
    img = cv2.resize(x,(img_w, img_h))
    
    X_final.append(img)
    
    Y = np.zeros((grid_y_axis,grid_x_axis,5,info))
    
    r_data_slice = r_data.loc[r_data['img_id'].isin([i])].reset_index(drop=True) # slicing as per current image id
    
    for j in range(len(r_data_slice)):
        
        xmin = r_data_slice['x'][j] * x_ratio
        ymin = r_data_slice['y'][j] * y_ratio
        
        xmax = (r_data_slice['x'][j] + r_data_slice['box_width'][j]) * x_ratio
        ymax = (r_data_slice['y'][j] + r_data_slice['box_height'][j]) * y_ratio
        
        obj_class = r_data_slice['obj_class'][j]
        
        w = (xmax - xmin)/img_w
        h = (ymax - ymin)/img_h

        x = (xmin + (xmax-xmin)/2)/img_w
        y = (ymin + (ymax-ymin)/2)/img_h
        
        x = x * grid_y_axis
        y = y * grid_x_axis
        
        idd = categories.tolist().index(obj_class)
        
                  
                               
        Y[int(y),int(x),idd,0] = 1
        Y[int(y),int(x),idd,1] = x - int(x)
        Y[int(y),int(x),idd,2] = y - int(y)
        Y[int(y),int(x),idd,3] = w
        Y[int(y),int(x),idd,4] = h
        
        balance_Y = y_encoded[categories.tolist().index(obj_class), :]
        
        Y[int(y),int(x),idd,5] = balance_Y[0]
        Y[int(y),int(x),idd,6] = balance_Y[1]
        Y[int(y),int(x),idd,7] = balance_Y[2]
        Y[int(y),int(x),idd,8] = balance_Y[3]
        Y[int(y),int(x),idd,9] = balance_Y[4]
        
        
            
        print(i, int(y), int(x), idd)
        print(Y[int(y),int(x),idd])
        
    
        
    Y_final.append(Y)
    print('==================================================')
print('++++++++++++++++++++++++++++++++++++++++++++++++++++++')
print(Y_final)
X = np.array(X_final) 
#X_final = []
Y = np.array(Y_final)
#Y_final = []

X = (X - 127.5)/127.5  # X normalising since pixels vary from 0 to 255
    
np.save('/home/scar3crow/Downloads/Data1/X_short-1.npy',X)
np.save('/home/scar3crow/Downloads/Data1/Y_short_1.npy',Y)


1a.jpg 2 6 0
[1.         0.01388889 0.91039427 0.68849206 0.21505376 0.
 0.         0.         0.         1.        ]
1a.jpg 11 10 1
[1.         0.33333333 0.390681   0.41666667 0.31541219 1.
 0.         0.         0.         0.        ]
1a.jpg 5 2 2
[1.         0.36111111 0.6702509  0.27380952 0.05734767 0.
 0.         1.         0.         0.        ]
1a.jpg 4 2 3
[1.         0.19444444 0.89247312 0.25       0.05376344 0.
 1.         0.         0.         0.        ]
1a.jpg 7 2 4
[1.         0.29166667 0.02508961 0.26388889 0.06451613 0.
 0.         0.         1.         0.        ]
2a.jpg 5 2 0
[1.         0.97011952 0.45348837 0.39641434 0.34883721 0.
 0.         0.         0.         1.        ]
2a.jpg 10 3 1
[1.         0.15139442 0.41860465 0.41832669 0.3255814  1.
 0.         0.         0.         0.        ]
2a.jpg 3 8 2
[1.         0.77091633 0.94767442 0.14143426 0.15697674 0.
 0.         1.         0.         0.        ]
2a.jpg 3 11 3
[1.         0.5876494  0.98837209 0.121

30a.jpg 7 6 0
[1.         0.9869403  0.40433213 0.71828358 0.16967509 0.
 0.         0.         0.         1.        ]
30a.jpg 10 5 1
[1.         0.10634328 0.23465704 0.49813433 0.19855596 1.
 0.         0.         0.         0.        ]
30a.jpg 4 2 2
[1.         0.23320896 0.92779783 0.25186567 0.09025271 0.
 0.         1.         0.         0.        ]
30a.jpg 4 11 3
[1.         0.42723881 0.97833935 0.16231343 0.08303249 0.
 1.         0.         0.         0.        ]
30a.jpg 13 2 4
[1.         0.20708955 0.26714801 0.2369403  0.08303249 0.
 0.         0.         1.         0.        ]
31a.jpg 8 2 0
[1.         0.68726592 0.58881579 0.26404494 0.37171053 0.
 0.         0.         0.         1.        ]
31a.jpg 8 7 1
[1.         0.4588015  0.24342105 0.30524345 0.30263158 1.
 0.         0.         0.         0.        ]
31a.jpg 12 2 2
[1.         0.88389513 0.36513158 0.29588015 0.04934211 0.
 0.         1.         0.         0.        ]
31a.jpg 12 9 3
[1.         0.93632959 0.6184

57a.jpg 3 6 0
[1.         0.66358595 0.01766784 0.53789279 0.19787986 0.
 0.         0.         0.         1.        ]
57a.jpg 11 9 1
[1.         0.8207024  0.60070671 0.36414048 0.30035336 1.
 0.         0.         0.         0.        ]
57a.jpg 5 2 2
[1.         0.48428835 0.59010601 0.23659889 0.04946996 0.
 0.         1.         0.         0.        ]
57a.jpg 4 2 3
[1.         0.43253235 0.84805654 0.23290203 0.0565371  0.
 1.         0.         0.         0.        ]
57a.jpg 8 8 4
[1.         0.3974122  0.75618375 0.15711645 0.04946996 0.
 0.         0.         1.         0.        ]
58a.jpg 3 6 0
[1.         0.59363296 0.28275862 0.5411985  0.20689655 0.
 0.         0.         0.         1.        ]
58a.jpg 11 9 1
[1.         0.79213483 0.63448276 0.37265918 0.28965517 1.
 0.         0.         0.         0.        ]
58a.jpg 5 2 2
[1.         0.26779026 0.79310345 0.2340824  0.04827586 0.
 0.         1.         0.         0.        ]
58a.jpg 5 2 3
[1.         0.28089888 0.1172413

In [12]:
Y.shape

(62, 14, 14, 5, 10)

In [13]:
X_train , X_val , Y_train , Y_val  = train_test_split(X,Y,train_size = 0.80, shuffle = True)

/home/scar3crow/Dropbox/WorkStation-Subrata/python/venv1/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [56]:
def my_model(input_shape):
    
    
    inp = Input(input_shape)
   
    model = MobileNetV2( input_tensor= inp , include_top=False, weights='imagenet')
    last_layer = model.output
    
#    last_3 = Flatten()(last_layer)
#    last_2 = Dense(640, activation = 'relu')(last_3)
#    last_1 = Dense(320, activation = 'relu')(last_2)
#    last_0 = Dense(7, activation = 'softmax')(last_1
    last_3 = UpSampling2D(2)(last_layer)

    last_2 = Conv2D(640,(3,3) , activation='relu' , padding='same')(last_3)
    
    last_1 = Conv2D(320,(1,1) , activation='relu' , padding='valid')(last_2)
    
    last_0 = Conv2D(50,(1,1) , activation='softmax', padding='valid')(last_1)
    
    
    
    final = Reshape((grid_y_axis,grid_x_axis,5,info))(last_0)
         
    model = Model(inp,final)
    
#    model = Model(inp,last_0)
    
    return model


In [57]:
input_size = (img_h,img_w,3)

my_invoice_model_2 = my_model(input_size)

print(my_invoice_model_2.summary())


/home/scar3crow/Dropbox/WorkStation-Subrata/python/venv1/lib/python3.5/site-packages/keras_applications/mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_4[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
____________________________________________________________________________________________

In [42]:
#define loss function

def yolo_loss_func(y_true,y_pred):
    
    # y_true : 14,14,5,10
    # y_pred : 14,14,5,10
    
    l_coords = 5.0
    l_noob = 0.5
    
    coords = y_true[:,:,:,:,0] * l_coords  # If the grid has a object - weitage increased by l_coords(5 times)
    noobs = (-1*(y_true[:,:,:,:,0] - 1)*l_noob) # If the grid does not have an object - penalised
    p_pred = y_pred[:,:,:,:,0]
    p_true = y_true[:,:,:,:,0]
    x_true = y_true[:,:,:,:,1]
    x_pred = y_pred[:,:,:,:,1]
    yy_true = y_true[:,:,:,:,2]
    yy_pred = y_pred[:,:,:,:,2]
    w_true = y_true[:,:,:,:,3]
    w_pred = y_pred[:,:,:,:,3]
    h_true = y_true[:,:,:,:,4]
    h_pred = y_pred[:,:,:,:,4]
    
    class_true = y_true[:,:,:,:,5:9]
    class_pred = y_pred[:,:,:,:,5:9]
    
    
    p_loss_absent = K.sum(K.square(p_pred - p_true)*noobs)
    p_loss_present = K.sum(K.square(p_pred - p_true)) * (coords / l_coords) # (coords/l_coords) added by me to 
                                                                            # exactly follow yolo loss formula
    x_loss = K.sum(K.square(x_pred - x_true)*coords)
    yy_loss = K.sum(K.square(yy_pred - yy_true)*coords)
    xy_loss = x_loss + yy_loss
    
    w_loss = K.sum(K.square(K.sqrt(w_pred) - K.sqrt(w_true))*coords)
    h_loss = K.sum(K.square(K.sqrt(h_pred) - K.sqrt(h_true))*coords)
    wh_loss = w_loss + h_loss
    
    class_loss = K.sum(K.sum(K.square(class_pred - class_true)) * (coords / l_coords)) # added by me
    
    
    loss = p_loss_absent + p_loss_present + xy_loss + wh_loss + class_loss
    
    return loss

In [58]:
opt = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

my_invoice_model_2.compile(loss='categorical_crossentropy' , optimizer=opt , metrics = ['accuracy'])

In [59]:
my_invoice_model_2.fit(X_train, Y_train, epochs= 10, batch_size = 4, validation_data=(X_val,Y_val))

Train on 49 samples, validate on 13 samples
Epoch 1/10
49/49 [==============================] - 31s 628ms/step - loss: 0.0374 - accuracy: 0.2353 - val_loss: 0.0335 - val_accuracy: 0.4441
Epoch 2/10
49/49 [==============================] - 18s 376ms/step - loss: 0.0329 - accuracy: 0.3548 - val_loss: 0.0329 - val_accuracy: 0.4204
Epoch 3/10
49/49 [==============================] - 19s 397ms/step - loss: 0.0312 - accuracy: 0.3340 - val_loss: 0.0318 - val_accuracy: 0.5108
Epoch 4/10
49/49 [==============================] - 18s 372ms/step - loss: 0.0302 - accuracy: 0.4471 - val_loss: 0.0315 - val_accuracy: 0.4240
Epoch 5/10
49/49 [==============================] - 20s 401ms/step - loss: 0.0296 - accuracy: 0.3633 - val_loss: 0.0307 - val_accuracy: 0.4801
Epoch 6/10
49/49 [==============================] - 21s 420ms/step - loss: 0.0291 - accuracy: 0.4490 - val_loss: 0.0310 - val_accuracy: 0.4533
Epoch 7/10
49/49 [==============================] - 21s 425ms/step - loss: 0.0289 - accuracy: 0.41

In [60]:
my_invoice_model_2.fit(X_train, Y_train, epochs= 10, batch_size = 4, validation_data=(X_val,Y_val))

Train on 49 samples, validate on 13 samples
Epoch 1/10
49/49 [==============================] - 19s 394ms/step - loss: 0.0284 - accuracy: 0.4391 - val_loss: 0.0306 - val_accuracy: 0.5352
Epoch 2/10
49/49 [==============================] - 19s 385ms/step - loss: 0.0282 - accuracy: 0.4966 - val_loss: 0.0308 - val_accuracy: 0.5225
Epoch 3/10
49/49 [==============================] - 20s 402ms/step - loss: 0.0282 - accuracy: 0.4697 - val_loss: 0.0303 - val_accuracy: 0.5188
Epoch 4/10
49/49 [==============================] - 20s 402ms/step - loss: 0.0281 - accuracy: 0.4637 - val_loss: 0.0305 - val_accuracy: 0.5138
Epoch 5/10
49/49 [==============================] - 19s 396ms/step - loss: 0.0280 - accuracy: 0.4666 - val_loss: 0.0303 - val_accuracy: 0.5113
Epoch 6/10
49/49 [==============================] - 21s 422ms/step - loss: 0.0280 - accuracy: 0.4689 - val_loss: 0.0301 - val_accuracy: 0.4839
Epoch 7/10
49/49 [==============================] - 20s 410ms/step - loss: 0.0279 - accuracy: 0.42

In [61]:
opt = Adam(lr=0.0002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

my_invoice_model_2.compile(loss='categorical_crossentropy' , optimizer=opt , metrics = ['accuracy'])

In [62]:
my_invoice_model_2.fit(X_train, Y_train, epochs= 10, batch_size = 4, validation_data=(X_val,Y_val))

Train on 49 samples, validate on 13 samples
Epoch 1/10
49/49 [==============================] - 28s 578ms/step - loss: 0.0281 - accuracy: 0.4572 - val_loss: 0.0318 - val_accuracy: 0.6690
Epoch 2/10
49/49 [==============================] - 19s 379ms/step - loss: 0.0285 - accuracy: 0.3969 - val_loss: 0.0327 - val_accuracy: 0.7052
Epoch 3/10
49/49 [==============================] - 19s 379ms/step - loss: 0.0283 - accuracy: 0.5023 - val_loss: 0.0345 - val_accuracy: 0.6500
Epoch 4/10
49/49 [==============================] - 19s 382ms/step - loss: 0.0281 - accuracy: 0.4306 - val_loss: 0.0341 - val_accuracy: 0.5911
Epoch 5/10
49/49 [==============================] - 20s 399ms/step - loss: 0.0279 - accuracy: 0.4810 - val_loss: 0.0363 - val_accuracy: 0.7456
Epoch 6/10
49/49 [==============================] - 20s 415ms/step - loss: 0.0278 - accuracy: 0.4622 - val_loss: 0.0365 - val_accuracy: 0.5729
Epoch 7/10
49/49 [==============================] - 18s 372ms/step - loss: 0.0278 - accuracy: 0.41

In [14]:
def _conv_block(inp, convs, skip=True):
    x = inp
    count = 0
    for conv in convs:
        if count == (len(convs) - 2) and skip:
            skip_connection = x
        count += 1
        if conv['stride'] > 1: x = ZeroPadding2D(((1,0),(1,0)))(x) # peculiar padding as darknet prefer left and top
        x = Conv2D(conv['filter'],
                   conv['kernel'],
                   strides=conv['stride'],
                   padding='valid' if conv['stride'] > 1 else 'same', # peculiar padding as darknet prefer left and top
                   name='conv_' + str(conv['layer_idx']),
                   use_bias=False if conv['bnorm'] else True)(x)
        if conv['bnorm']: x = BatchNormalization(epsilon=0.001, name='bnorm_' + str(conv['layer_idx']))(x)
        if conv['leaky']: x = LeakyReLU(alpha=0.1, name='leaky_' + str(conv['layer_idx']))(x)
    return add([skip_connection, x]) if skip else x
 
def make_yolov3_model():
    input_image = Input(shape=(224, 224, 3))
    # Layer  0 => 4
    x = _conv_block(input_image, [{'filter': 32, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 0},
                                  {'filter': 64, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 1},
                                  {'filter': 32, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 2},
                                  {'filter': 64, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 3}])
    # Layer  5 => 8
    x = _conv_block(x, [{'filter': 128, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 5},
                        {'filter':  64, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 6},
                        {'filter': 128, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 7}])
    # Layer  9 => 11
    x = _conv_block(x, [{'filter':  64, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 9},
                        {'filter': 128, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 10}])
    # Layer 12 => 15
    x = _conv_block(x, [{'filter': 256, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 12},
                        {'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 13},
                        {'filter': 256, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 14}])
    # Layer 16 => 36
    for i in range(7):
        x = _conv_block(x, [{'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 16+i*3},
                            {'filter': 256, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 17+i*3}])
    skip_36 = x
    # Layer 37 => 40
    x = _conv_block(x, [{'filter': 512, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 37},
                        {'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 38},
                        {'filter': 512, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 39}])
    # Layer 41 => 61
    for i in range(7):
        x = _conv_block(x, [{'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 41+i*3},
                            {'filter': 512, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 42+i*3}])
    skip_61 = x
    # Layer 62 => 65
    x = _conv_block(x, [{'filter': 1024, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 62},
                        {'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 63},
                        {'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 64}])
    # Layer 66 => 74
    for i in range(3):
        x = _conv_block(x, [{'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 66+i*3},
                            {'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 67+i*3}])
    # Layer 75 => 79
    x = _conv_block(x, [{'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 75},
                        {'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 76},
                        {'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 77},
                        {'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 78},
                        {'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 79}], skip=False)
    # Layer 80 => 82
    yolo_82 = _conv_block(x, [{'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 80},
                              {'filter':  255, 'kernel': 1, 'stride': 1, 'bnorm': False, 'leaky': False, 'layer_idx': 81}], skip=False)
    # Layer 83 => 86
    x = _conv_block(x, [{'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 84}], skip=False)
    x = UpSampling2D(2)(x)
    x = concatenate([x, skip_61])
    # Layer 87 => 91
    x = _conv_block(x, [{'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 87},
                        {'filter': 512, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 88},
                        {'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 89},
                        {'filter': 512, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 90},
                        {'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 91}], skip=False)
    # Layer 92 => 94
    yolo_94 = _conv_block(x, [{'filter': 512, 'kernel': 3, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 92},
                              {'filter': 50, 'kernel': 1, 'stride': 1, 'bnorm': False, 'leaky': False, 'layer_idx': 93}], skip=False)
    
    final = Reshape((grid_y_axis,grid_x_axis,5,info))(yolo_94)
    model = Model(input_image, [final])
    return model
 
 
# define the model
model = make_yolov3_model()


In [15]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv_0 (Conv2D)                 (None, 224, 224, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
bnorm_0 (BatchNormalization)    (None, 224, 224, 32) 128         conv_0[0][0]                     
__________________________________________________________________________________________________
leaky_0 (LeakyReLU)             (None, 224, 224, 32) 0           bnorm_0[0][0]                    
____________________________________________________________________________________________

In [16]:
opt = Adam(lr=0.0002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(loss='categorical_crossentropy' , optimizer=opt , metrics = ['accuracy'])

In [ ]:
model.fit(X_train, Y_train, epochs= 10, batch_size = 4, validation_data=(X_val,Y_val))

Train on 49 samples, validate on 13 samples
Epoch 1/10
16/49 [========>.....................] - ETA: 4:32 - loss: 0.1182 - accuracy: 0.0465

In [17]:
model.fit(X_train, Y_train, epochs= 10, batch_size = 4, validation_data=(X_val,Y_val))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 49 samples, validate on 13 samples
Epoch 1/10
49/49 [==============================] - 64s 1s/step - loss: 0.1280 - accuracy: 0.0686 - val_loss: 0.1106 - val_accuracy: 0.0220
Epoch 2/10
49/49 [==============================] - 54s 1s/step - loss: 0.1239 - accuracy: 0.0589 - val_loss: 0.1424 - val_accuracy: 0.0010
Epoch 3/10
49/49 [==============================] - 56s 1s/step - loss: 0.1211 - accuracy: 0.0596 - val_loss: 0.1391 - val_accuracy: 0.0031
Epoch 4/10
49/49 [==============================] - 58s 1s/step - loss: 0.1516 - accuracy: 0.0554 - val_loss: 0.1365 - val_accuracy: 0.0094
Epoch 5/10
49/49 [==============================] - 57s 1s/step - loss: 0.1367 - accuracy: 0.0423 - val_loss: 0.1196 - val_accuracy: 0.0000e+00
Epoch 6/10
49/49 [==============================] - 58s 1s/step - loss: 0.1252 - accuracy: 0.0432 - val_loss: 0.0886 - val_accuracy: 4.7096e-04
Epoch 7/10
49

In [18]:
opt = Adam(lr=0.0004, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(loss='categorical_crossentropy' , optimizer=opt , metrics = ['accuracy'])

In [19]:
model.fit(X_train, Y_train, epochs= 10, batch_size = 4, validation_data=(X_val,Y_val))

Train on 49 samples, validate on 13 samples
Epoch 1/10
49/49 [==============================] - 64s 1s/step - loss: 0.1273 - accuracy: 0.0951 - val_loss: 0.1098 - val_accuracy: 0.0163
Epoch 2/10
49/49 [==============================] - 56s 1s/step - loss: 0.1179 - accuracy: 0.0993 - val_loss: 0.1042 - val_accuracy: 0.1949
Epoch 3/10
49/49 [==============================] - 56s 1s/step - loss: 0.1186 - accuracy: 0.1451 - val_loss: 0.1336 - val_accuracy: 0.0564
Epoch 4/10
49/49 [==============================] - 57s 1s/step - loss: 0.1209 - accuracy: 0.1421 - val_loss: 0.1155 - val_accuracy: 0.0043
Epoch 5/10
49/49 [==============================] - 58s 1s/step - loss: 0.1169 - accuracy: 0.1695 - val_loss: 0.1367 - val_accuracy: 0.0148
Epoch 6/10
49/49 [==============================] - 57s 1s/step - loss: 0.1189 - accuracy: 0.1532 - val_loss: 0.1219 - val_accuracy: 0.0133
Epoch 7/10
49/49 [==============================] - 57s 1s/step - loss: 0.1228 - accuracy: 0.1558 - val_loss: 0.1254